In [1]:
import os
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, UpSampling2D

# Define the paths to the clear and foggy image folders -- CHANGE THESE PATHS TO YOUR OWN
clear_folder = 'C:\\Users\\maraf\\OneDrive\\Desktop\\ZippedGift\\ZippedGift\\OG_ModelBasedOnOurOwnDataset\\Foggy'
foggy_folder = 'C:\\Users\\maraf\\OneDrive\\Desktop\\ZippedGift\\ZippedGift\\OG_ModelBasedOnOurOwnDataset\\Clear'

# Load the clear images
clear_images = []
for filename in os.listdir(clear_folder):
    if filename.endswith((".png", ".jpg")):  # Check for valid image file extensions
        clear_img = cv2.imread(os.path.join(clear_folder, filename))
        if clear_img is not None:  # Check if the image was loaded successfully
            clear_images.append(clear_img)

# Load the foggy images
foggy_images = []
for filename in os.listdir(foggy_folder):
    if filename.endswith((".png", ".jpg")):  # Check for valid image file extensions
        foggy_img = cv2.imread(os.path.join(foggy_folder, filename))
        if foggy_img is not None:  # Check if the image was loaded successfully
            foggy_images.append(foggy_img)

# Convert the lists of images to numpy arrays
clear_images = np.array(clear_images)
foggy_images = np.array(foggy_images)

# Get the size of the images to determine the dimensions
image_height, image_width, num_channels = foggy_images[0].shape
print("Image dimensions:", image_height, image_width, num_channels)

# Resize the images to the desired size
resized_clear_images = []
resized_foggy_images = []
for clear_img, foggy_img in zip(clear_images, foggy_images):
    resized_clear_img = cv2.resize(clear_img, (image_width, image_height))
    resized_foggy_img = cv2.resize(foggy_img, (image_width, image_height))
    resized_clear_images.append(resized_clear_img)
    resized_foggy_images.append(resized_foggy_img)

# Convert the lists of resized images to numpy arrays
resized_clear_images = np.array(resized_clear_images)
resized_foggy_images = np.array(resized_foggy_images)

# Normalize pixel values to range between 0 and 1
resized_clear_images = resized_clear_images.astype('float32') / 255.0
resized_foggy_images = resized_foggy_images.astype('float32') / 255.0

# Define the CNN model architecture
model = Sequential()
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', input_shape=(image_height, image_width, num_channels)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
# model.add(UpSampling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(num_channels, (3, 3), activation='sigmoid', padding='same'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(resized_foggy_images, resized_clear_images, epochs=10, batch_size=16)

# Save the trained model
model.save("foggy_to_clear_model.h5")


Image dimensions: 480 640 3
Epoch 1/10
